In [ ]:
#### Load the plot setup for thesis quality plots
from mpl_latex_settings import create_latex_setup
config = "/Users/tmenne/git/thesis/inc/mpl_latex_config.json"
latex_setup = create_latex_setup(config, interactive=True)

In [ ]:
import matplotlib as mpl
mpl.rcParams["figure.figsize"]

In [ ]:
from __future__ import division, print_function
import os
import json
import gzip
import healpy as hp
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as sci
import scipy.stats as scs
import scipy.optimize as sco
from scipy.ndimage.filters import gaussian_filter
from matplotlib import gridspec
from matplotlib.colors import LogNorm, LinearSegmentedColormap
from astropy.time import Time as astrotime

import tdepps.utils as tdu
from mypyscripts.general import arr2str
from mypyscripts.stats import (sigma2prob, prob2sigma, percentile_nzeros,
                               cdf_nzeros)

import mypyscripts.plots as mplt
from mypyscripts.plots import (skymap, plot_healpy_map, mollview,
                               render_healpymap)
#
from _loader import loader as LOADER
from _paths import PATHS_ORIG as PATHS  # Use the original 22 HESE sources

# Make some globals
SECINDAY = 24. * 60. * 60.
RNDGEN = np.random.RandomState(42439462)
loader = LOADER(PATHS, verb=False)

print("Paths are:\n", PATHS)

def save_fig(fig, path, **save_args):
    path = os.path.abspath(path)
    dirname = os.path.dirname(path)
    if not os.path.isdir(dirname):
        os.makedirs(dirname)   
    fig.savefig(path, **save_args)
    
def key2label(key, short=False):
    """ Makes 'IC86, 2012--2014' from 'IC86_2012-2104' """
    if short:  # Abbreviate 20YY -> 'YY
        return ", ".join(map(lambda s: s.replace("-", "--").replace("20", "'"),
                             key.split("_")))
    return ", ".join(map(lambda s: s.replace("-", "--"), key.split("_")))

def trunc_cmap(cmap, arr):
    """
    Adapted from: https://stackoverflow.com/questions/18926031
    Makes a new colormap with original values selected from arr in [0, 1].
    """
    vmin, vmax = np.amin(arr), np.amax(arr)
    new_cmap = LinearSegmentedColormap.from_list(
        'trunc({}_{:.2f}_{:.2f})'.format(cmap.name, vmin, vmax), cmap(arr))
    return new_cmap

# Chapter: Datasets

## Count number of events originally in the data sets

In [ ]:
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC79b_exp.npy")
print("IC79: ", len(exp))
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86_exp.npy")
print("IC86, 2011: ", len(exp))

exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86-2012_exp_v2.npy")
print("IC86, 2012: ", len(exp))
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86-2013_exp_v2.npy")
print("IC86, 2013: ", len(exp))
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86-2014_exp_v2.npy")
print("IC86, 2014: ", len(exp))

exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/SplineMPEmax.MuEx.IC86-2015.npy")
print("IC86, 2015: ", len(exp))

## Print out source info for table

In [ ]:
src_list = loader.source_list_loader("all")

src_decs = []
src_ras = []
src_mjds = []
for name, sl in sorted(src_list.items()):
    src_decs = np.concatenate([src_decs, [sli["dec"] for sli in sl]])
    src_ras = np.concatenate([src_ras, [sli["ra"] for sli in sl]])
    src_mjds = np.concatenate([src_mjds, [sli["mjd"] for sli in sl]])

In [ ]:
print(arr2str(np.rad2deg(src_decs), fmt="{:.2f}"))
print("")
print(arr2str(np.rad2deg(src_ras), fmt="{:.2f}"))
print("")
print(arr2str(src_mjds, fmt="{:.2f}"))

## Show HESE / no HESE in MC

The region high energy region from the northern sky should be investigated.
In true variables it looks like statistics, but in proxies it blows up enormuously.
Should the main effect come from the souther high energy sky?

In [ ]:
def remove_hese_from_mc(mc, heseids):
    """
    Mask all values in ``mc`` that have the same run and event ID combination
    as in ``heseids``.

    Parameters
    ----------
    mc : record-array
        MC data, needs names ``'Run', 'Event'``.
    heseids : dict or record-array
        Needs names / keys ``'run_id', 'event_id``.

    Returns
    -------
    is_hese_like : array-like, shape (len(mc),)
        Mask: ``True`` for each event in ``mc`` that is HESE like.
    """
    # Make combined IDs to easily match against HESE IDs with `np.isin`
    factor_mc = 10**np.ceil(np.log10(np.amax(mc["Event"])))
    _evids = np.atleast_1d(heseids["event_id"])
    factor_hese = 10**np.ceil(np.log10(np.amax(_evids)))
    factor = max(factor_mc, factor_hese)

    combined_mcids = (factor * mc["Run"] + mc["Event"]).astype(int)
    assert np.all(combined_mcids > factor)  # Is int overflow a thing here?

    _runids = np.atleast_1d(heseids["run_id"])
    combined_heseids = (factor * _runids + _evids).astype(int)
    assert np.all(combined_heseids > factor)

    # Check which MC event is tagged as HESE like
    is_hese_like = np.isin(combined_mcids, combined_heseids)
    print("  Found {} / {} HESE like events in MC".format(np.sum(is_hese_like),
                                                          len(mc)))
    return is_hese_like

def _get_weights(ow, trueE):
    """ Common model to weight to event rate """ 
    lt = 365. * SECINDAY
    return ow * tdu.power_law_flux(trueE, E0=1e5,
                                   phi0=1.01e-18, gamma=2.19) * lt

name2skylab = {
    "IC79" : "IC79b_corrected_MC.npy",
    "IC86_2011" : "IC86_corrected_MC.npy",
    "IC86_2012-2014" : "IC86-2012_corrected_MC_v2.npy",
    "IC86_2015" : "SplineMPEmax.MuEx.MC.npy",
}
name2idx = {
    "IC79" : "IC79.json.gz",
    "IC86_2011" : "IC86_2011.json.gz",
    "IC86_2012-2014" : "IC86_2012-2015.json.gz",
    "IC86_2015" : "IC86_2012-2015.json.gz",    
}

In [ ]:
proxy = False  # Switch False | True to see plots in trueE, trueDec, vs. proxies
if proxy:
    _sindec = lambda mc: np.sin(mc["dec"])
    _logE = lambda mc: mc["logE"]
    _logE_label = r"$\log_{10}(E_\mathrm{proxy} / \mathrm{GeV})$"
    _sindec_label = r"$\sin(\delta_\mathrm{proxy})$"
    _info = "_proxy"
else:
    _sindec = lambda mc: np.sin(mc["trueDec"])
    _logE = lambda mc: np.log10(mc["trueE"])
    _logE_label = r"$\log_{10}(E_\nu / \mathrm{GeV})$"
    _sindec_label = r"$\sin(\delta_\nu)$"
    _info = ""

for sample_name in name2skylab.keys()[:]:
    print("# {}".format(sample_name))
    # Load full skylab data
    _mc = np.load(os.path.join(PATHS.skylab_data, name2skylab[sample_name]))

    # Filter HESE events
    _path = os.path.join(PATHS.local, "check_hese_mc_ids",
                         name2idx[sample_name])
    heseids = json.load(gzip.open(_path))
    is_hese_mask = remove_hese_from_mc(_mc, heseids)
    _mc_hese_only = _mc[is_hese_mask]
    # Get weights
    w_all = _get_weights(_mc["ow"], _mc["trueE"])
    w_hese_only = _get_weights(_mc_hese_only["ow"], _mc_hese_only["trueE"])

    # Plot filtered out HESE events
    try:
        fs = latex_setup.make_figsize(scale=0.95, ratio=2.5)
    except:
        fs = (10, 4)
    fig, (axl, axr) = plt.subplots(1, 2, sharey=True, figsize=fs)
    
    # Make histograms for contourf
    bins = [np.linspace(-1, 1, 40), np.linspace(2, 9, 40)]
            # np.linspace(np.amin(_logE(_mc)), np.amax(_logE(_mc)), 40)]
    h, _, _ = np.histogram2d(_sindec(_mc), _logE(_mc), normed=False,
                             bins=bins, weights=w_all)
    h_nh, _, _ = np.histogram2d(_sindec(_mc_hese_only), _logE(_mc_hese_only),
                                bins=bins, weights=w_hese_only, normed=False)
    vmin = 10**np.floor(np.amin(np.log10(h[h>0])))
    vmax = 10**np.ceil(np.amax(np.log10(h[h>0])))

    # Plot whole MC left, weights normed to 365 days
    h_pad = np.pad(h.T, mode="edge", pad_width=[(0, 1), (0, 1)])
    levels = np.logspace(-4, 0, 11)
    img = axl.contourf(bins[0], bins[1], h_pad, levels,
                       cmap="Greys", norm=LogNorm())
    cbar = fig.colorbar(img, ax=axl, ticks=levels[::2])
    cbar.ax.set_yticklabels([])
    cbar.ax.set_yticks(levels[::2])
#     cbar.ax.set_yticklabels(["$10^{{{:.0f}}}$".format(li)
#                              for li in np.log10(levels[::2])])
    
    # Right plot only cut-out HESE, weights normed to 365 days
    h_nh_pad = np.pad(h_nh.T, mode="edge", pad_width=[(0, 1), (0, 1)])
    img = axr.contourf(bins[0], bins[1], h_nh_pad, levels,
                       cmap="Greys", norm=LogNorm())
    cbar = fig.colorbar(img, ax=axr, ticks=levels[::2])
    cbar.ax.set_yticklabels(["$10^{{{:.0f}}}$".format(li)
                             for li in np.log10(levels[::2])])
    cbar.set_label("Differenial no. of events")

#     axl.set_title("No. events for whole MC: {:.2f}".format(np.sum(w_all)))
#     axr.set_title("No. events for whole HESE-like MC: " +
#                   "{:.2f}".format(np.sum(w_hese_only)))
    for ax in (axl, axr):
        ax.set_xlabel(_sindec_label)
        ax.set_ylim(bins[1][0], bins[1][-1])
        ax.grid(ls=":")
    axl.set_ylabel(_logE_label)
    
    fig.tight_layout(w_pad=-0.2)
    
    save_fig(fig, "plots/datasets/mc_no_hese_{}.pdf".format(sample_name),
             bbox_inches="tight")
#     save_fig(fig, "plots/sample/mc_no_hese_{}.pgf".format(sample_name),
#              bbox_inches="tight")

    plt.show()

Old plots compared ratios

In [ ]:
fig, ax = plt.subplots(1, 1)

ratio_pad = np.pad(ratio, pad_width=[(0, 1), (0, 1)], mode="edge")
levels = np.linspace(0.6, 1., 9)
img = ax.contourf(bins[0], bins[1], ratio_pad.T, levels,
                  cmap="Greys_r", extend="min")
fig.colorbar(img, ax=ax)

plt.show()


# Or compare to pcolormesh
fig, ax = plt.subplots(1, 1)

img = ax.pcolormesh(bins[0], bins[1], ratio.T, cmap="Greys_r", vmin=0.75)
fig.colorbar(img, ax=ax)

plt.show()

## Source positions

In [ ]:
src_list = loader.source_list_loader("all")
src_maps = {k: loader.source_map_loader(sl) for k, sl in src_list.items()}

In [ ]:
try:
    fs = latex_setup.make_figsize(ratio=2.5, scale=1.)
except:
    fs = (6, 4)

One version with the reco maps

In [ ]:
src_map_sum = np.sum(
    [np.sum(srcm, axis=0) for srcm in src_maps.values()], axis=0)

skym = skymap()
fig, ax = skym.figure(
    None, None, proj="mollweide", grid=True, gal_plane=True,
    tex=True, grid_move_to_back=False, gal_plane_label=None,
    figsize=fs)
# Don't overdo with dpi and levels when using contourf -> large files!
dpi = 72
fig.set_dpi(val=dpi)  # To set the pcolormesh xx, yy dimension

# Old plot with pcolormesh and rasterized
# plot_healpy_map(ax=ax, m=src_map_sum, cmap="Greys", rasterize=True)
levels = np.round(scs.chi2.sf(np.arange(1., 3. + 1., 1.)[::-1]**2, df=2) *
                  np.amax(src_map_sum))

cmap = trunc_cmap(plt.cm.Greys, np.r_[0., np.linspace(0.6, 1.0, 3)])
plot_healpy_map(ax=ax, m=src_map_sum, cmap=cmap, extend="both",
                levels=levels, draw_contour=True)
# plt.legend(loc="upper right")

save_fig(plt.gcf(), "plots/datasets/hese_events_reco_landscape_skymap.pdf",
         bbox_inches="tight")

plt.show()

And one version with the sources per sample

In [ ]:
skym = skymap()
fig, ax = skym.figure(
    None, None, proj="mollweide", grid=True, gal_plane=True,
    tex=True, grid_move_to_back=False, figsize=fs, gal_plane_label=None)

marker = ["x", "d", "+", "s"]
ms = [5, 3, 5, 3]
colors = 4 * ["k"]
for i, (name, srcs) in enumerate(sorted(src_list.items())):
    src_ra = np.array([src["ra"] for src in srcs])
    src_dec = np.array([src["dec"] for src in srcs])
    
    short = False
    if name == "IC86_2012-2014":
        short = True
    
    x, y = skym.EquCoordsToMapCoords(dec=src_dec, ra=src_ra)   
    ax.plot(x, y, color=colors[i], marker=marker[i],
            ls="", ms=ms[i], label=key2label(name, short=short))
    
leg = ax.legend(
    loc="upper left", ncol=1,
    # bbox_to_anchor=(0.5, 0.925),
    # bbox_transform=plt.gcf().transFigure,
    prop={"size": latex_setup.latex_settings["legend.fontsize"] - 3},
    framealpha=1.)

save_fig(plt.gcf(), "plots/datasets//hese_events_per_sample_skymap.pdf",
         bbox_inches="tight")
# save_fig(plt.gcf(), "plots/srcs/hese_events_per_sample_skymap.pdf",
#          bbox_inches="tight", bbox_extra_artists=[leg], pad_inches=0.2)

plt.show()

## Show zoomed version before / after smoothing

In [ ]:
%%time
# Load local, unsmoothed maps
files = sorted(glob("/Users/tmenne/Downloads/hese_transient_stacking_data/" +
                    "hese_scan_maps_local/*.json.gz"))
with gzip.open(files[0]) as fp:
    src_dict = json.load(fp)
    local_map = np.array(src_dict["map"])

In [ ]:
# Render the map around the minimum
res = (400, 300)

bf_azi, bf_zen = src_dict["bf_loc"]["azi"], src_dict["bf_loc"]["zen"]
off = np.deg2rad(5)
bounds = [[bf_azi - off, bf_azi + off], [bf_zen - off, bf_zen + off]]
local, cx, cy = mplt.render_healpymap(local_map, npix=res, bounds=bounds)
# Make -log10(DeltalogLLH), so levels show sigma contours
# when approx. gaussian like
delta_local = np.amax(local) - local

# Smooth it
pdf_map = pdf_map = np.exp(local_map - np.amax(local_map))
# Smooth with a gaussian kernel
pdf_map = hp.smoothing(map_in=pdf_map, sigma=np.deg2rad(1.), verbose=False)
# Healpy smoothing may produce numerical erros, so fix them after smoothing
pdf_map[pdf_map < 0.] = 0.
smoothed = np.log(pdf_map)
# Render a new version
smoothed, cx, cy = mplt.render_healpymap(smoothed, npix=res, bounds=bounds)
delta_smoothed = np.amax(smoothed) - smoothed

In [ ]:
try:
    fs = latex_setup.make_figsize(scale=0.9, ratio=2.5)
except:
    fs = (5.2, 2.)
# fig, (axl ,axr) = plt.subplots(1, 2, figsize=fs, sharey=True)
fig = plt.figure(figsize=fs)
axl =  fig.add_axes([0.1, 0.1, 0.4, 0.9])
axr =  fig.add_axes([0.55, 0.1, 0.4, 0.9])
axcbar =  fig.add_axes([0.975, 0.1, 0.025, 0.9])

# Local coords azi, zen as stored in map
mx, my = map(lambda b: np.rad2deg(0.5 * (b[:-1] + b[1:])), [cx, cy])
xx, yy = np.meshgrid(mx, my)
levels = - scs.chi2.logsf(np.arange(0., 3.5, 0.5)**2, df=2)

# Smooth the larger contour a bit
delta_smoothed = gaussian_filter(delta_smoothed, sigma=4.)

axl.contourf(xx, yy, delta_local, levels, cmap="Greys_r", extend="max")
img = axr.contourf(xx, yy, delta_smoothed, levels, cmap="Greys_r", extend="max")
cbar = fig.colorbar(img, cax=axcbar)
cbar.set_ticks(levels[::2])
cbar.set_label(r"$-\log_{10}(\mathrm{PDF})$")

xticks = np.array([334, 336, 338, 340, 342])
yticks = np.array([54, 56, 58, 60, 62, 64])
for ax in (axl, axr):
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_xlim(-1 + xticks[0], xticks[-1] + 1)
    ax.set_ylim(-1 + yticks[0], yticks[-1] + 1)
    ax.set_xlabel(r"Azimuth $\phi$ in $^\circ$")
    ax.grid(ls=":")

axr.set_yticks(axl.get_yticks())
axr.set_yticklabels([])
axl.set_ylabel(r"Zenith $\theta$ in $^\circ$")

fig.tight_layout()
save_fig(fig, "plots/datasets/unsmoothed_vs_smoothed.pdf", bbox_inches="tight")
plt.show()

## Show whole map raw smoothed and truncated

In [ ]:
%%time
# Load untruncaed maps
files = sorted(glob("/Users/tmenne/Downloads/hese_transient_stacking_data/" +
                    "rawout_original_hese/hese_scan_maps/*.json.gz"))
# Just pick a single map here
with gzip.open(files[0]) as fp:
    src_dict = json.load(fp)
    untruncated = np.array(src_dict["map"])

In [ ]:
res = (400, 300)

untrunc, cx, cy = mplt.render_healpymap(untruncated, npix=res)
# Make -log10(PDF), so levels show sigma contours when approx. gaussian like
m = (untrunc > 0.)
untrunc = np.log10(untrunc)
untrunc = np.amax(untrunc) - untrunc

# Truncate map at six sigma
_, _, idx = tdu.get_pixel_in_sigma_region(pdf_map=untruncated, sigma=6.)
truncated = np.zeros_like(untruncated)
truncated[idx] = untruncated[idx]
# And make a rendered -log10(PDF) version.
trunc, cx, cy = mplt.render_healpymap(truncated, npix=res)
m = (trunc > 0.)
trunc = np.log10(trunc)
trunc = np.amax(trunc) - trunc

In [ ]:
# Untruncated on the left truncated on the right
try:
    fs = latex_setup.make_figsize(scale=0.9, ratio=2.5)
except:
    fs = (5.2, 2.)
# fig, (axl ,axr) = plt.subplots(1, 2, figsize=fs, sharey=True)
fig = plt.figure(figsize=fs)
axl =  fig.add_axes([0.1, 0.1, 0.4, 0.9])
axr =  fig.add_axes([0.55, 0.1, 0.4, 0.9])
axcbar =  fig.add_axes([0.975, 0.1, 0.025, 0.9])

mx, my = map(lambda b: np.rad2deg(0.5 * (b[:-1] + b[1:])), [cx, cy])
# Convert to equ. with ra from right to left in hours
mx = 24. - mx / 360. * 24
my = 90. - my
xx, yy = np.meshgrid(mx, my)
levels = - scs.chi2.logsf(np.arange(0., 7, 1)**2, df=2)

axl.contourf(xx, yy, untrunc, levels, cmap="Greys_r", extend="max")
img = axr.contourf(xx, yy, trunc, levels, cmap="Greys_r", extend="max")
cbar = fig.colorbar(img, cax=axcbar)
cbar.set_ticks(levels[::2])
cbar.set_label(r"$-\log_{10}(\mathrm{PDF})$")

xticks = np.linspace(0, 24, 7)    # Show as hours from right to left 
yticks = np.linspace(-90, 90, 7)  # Show as equ. coords. in [-90, +90]
xlabels = [r"${:.0f}$".format(xi) for xi in xticks][::-1]
ylabels = [r"${:.0f}$".format(yi) for yi in yticks]
for ax in (axl, axr):
    ax.set_xticks(xticks)
    ax.set_xticklabels(xlabels)
    ax.set_xlabel(r"Right-ascension $\alpha$ in hours")
    ax.set_yticks(yticks)

axl.set_yticklabels(ylabels)
axr.set_yticklabels([])
axl.set_ylabel(r"Declination $\delta$ in $^\circ$")
axr.grid(ls=":")

plt.tight_layout()
save_fig(fig, "plots/datasets/untruncated_vs_truncated.pdf", bbox_inches="tight")
plt.show()

## Sindec distribution for true dec

In [ ]:
mc = loader.mc_loader("all")

In [ ]:
bins = np.linspace(-1, 1, 100)
mids = 0.5 * (bins[:-1] + bins[1:])
Hz2mhZ = 1e3
gammas = [2.5, 2.19, 2.]
fmts = [r"$E^{{-{:.1f}}}$", r"$E^{{-{:.2f}}}$", r"$E^{{-{:.0f}}}$"]
labels = [f.format(g) for f, g in zip(fmts, gammas)]
cs = ["0.6", "0.3", "0."]
phi0 = 1e-18
E0 = 1e5

for name in mc.keys():
    mc_i = mc[name]
    for i, gamma in enumerate(gammas):  # phi0 * (E/E0)**-gamma
        # Rate in mHz
        w = Hz2mhZ * phi0 * mc_i["ow"] * (mc_i["trueE"] / E0)**-gamma
        h, b = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w,
                            density=False)
        err, _ = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w**2,
                              density=False)
        plt.plot(b, np.r_[h[0], h], drawstyle="steps-pre", c=cs[i], lw=2,
                 label=labels[i])
        plt.errorbar(
            mids, h, yerr=err, fmt=",", color=cs[i], zorder=-10, lw=1)

    plt.xlabel(r"$\sin(\delta_\nu)$")
    plt.ylabel("Rate per bin in mHz")
    plt.title("Sample {}".format(key2label(name)))

    plt.yscale("log")
    plt.xlim(-1, 1)
    plt.ylim(5e-7, 3e-4)
    plt.legend(loc="upper left")
    
    save_fig(plt.gcf(), "plots/datasets/true_sindec_{}.pdf".format(name),
             bbox_inches="tight")
    
    plt.show()

## Effective areas for the PS sample

In [ ]:
mc = loader.mc_loader("all")

In [ ]:
cm2tom2 = 1e4
gevtotev = 1e3

e_bins = np.logspace(2, 9, 100)  # in GeV
x = np.logspace(e_bins[0], e_bins[-1], 500)
e_mids = 0.5 * (e_bins[:-1] + e_bins[1:])
decs_lo = np.deg2rad([-90, 30, -5, -90])
decs_hi = np.deg2rad([ 90, 90, 30,  -5])

for name, mc_i in mc.items():
    mc_i = mc[name]
    Enu = mc_i["trueE"]    # in GeV
    dec = mc_i["trueDec"]  # in rad
    w = mc_i["ow"]

    # Make eff. area in three decl. regions and full sky
    ls = ["-", "-", "-", "-"]
    cs = ["#353132", "0.3", "0.5", "0.75"]
    _s = r"$\SI{{{:.0f}}}{{\degree}} \leq \delta \leq \SI{{{:.0f}}}{{\degree}}$"
    labels = [_s.format(*np.rad2deg(dec_rng))
              for dec_rng in zip(decs_lo, decs_hi)]
    labels[0] = "All-sky"

    for i, (dec_lo, dec_hi) in enumerate(zip(decs_lo, decs_hi)):
        mask = np.logical_and(dec >= dec_lo, dec < dec_hi)
        Enu_i, w_i = Enu[mask], w[mask]
        h, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i, normed=False)
        solid_angle = 2 * np.pi * (np.sin(dec_hi) - np.sin(dec_lo))
        norm = np.diff(e_bins) * solid_angle * cm2tom2
        err, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i**2, normed=False)
        h = h / norm
        err = np.sqrt(err) / norm
        
        # truncate manually for pgf
        ymin = 1e-3
        h[h < ymin] = ymin / 10.

        plt.plot(e_bins / gevtotev, np.r_[h[0], h], drawstyle="steps-pre",
                 c=cs[i], label=labels[i], ls=ls[i])
        plt.errorbar(e_mids / gevtotev, h, yerr=err, fmt=",",
                     color=cs[i], zorder=-10, lw=1)
        
    plt.xscale("log")
    plt.yscale("log")
    plt.xlim(e_bins[0] / gevtotev, e_bins[-1] / gevtotev)
    plt.ylim(ymin, 1e4)
    plt.grid()

    plt.xlabel(r"True neutrino energy $E_\nu$ in $\si{\TeV}$")
    plt.ylabel(r"Eff. area $A_\mathrm{eff}^{\nu_\mu+\bar{\nu}_\mu}$ in " +
               r"$\si{\m\squared}$")
    plt.title("Sample {}".format(key2label(name)))
    plt.legend()

    save_fig(plt.gcf(), "plots/datasets/eff_area_{}.pdf".format(name),
             bbox_inches="tight")
    
    plt.show()

# Chapter: Time Independent Analysis

# BG only TS

In [ ]:
%%time
bg_pdfs = loader.bg_pdf_loader(idx="all")
tw_ids = sorted(bg_pdfs.keys())

Scan best threshold for scan plots

In [ ]:
%%time
threshs = []
res = []

for tw_id in tw_ids:
    print("# Time window {}".format(tw_id))      
    # Scan in a range with still good statistics, but leave the really good
    # statistics part to the empirical PDF
    bg_pdfs[tw_id].fit_thresh(np.amax(bg_pdfs[tw_id].data))
    lo, hi = bg_pdfs[tw_id].ppf(q=100. * sigma2prob([3., 5.5]))
    threshs.append(np.arange(lo, hi, 0.1))
    # Best fit: KS test p-value is larger than `pval_thresh` the first time
    pval_thresh = 0.5
    res_i = tdu.scan_best_thresh(
        bg_pdfs[tw_id], threshs[-1], pval_thresh=pval_thresh)
    best_thresh, best_idx, pvals, scales = res_i
    res.append(res_i)

    best_thresh_sigma = prob2sigma(bg_pdfs[tw_id].cdf(best_thresh))
    print("Best thresh at TS = {:.2f}, {:.2f} sigma".format(
        best_thresh, best_thresh_sigma[0]))

In [ ]:
# For the other notebook...
idx = np.array([res_i[1] for res_i in res])
locs = np.array([res_i[0] for res_i in res])
scales = np.array([res_i[3][idx[i]] for i, res_i in enumerate(res)])

In [ ]:
for tw_id in tw_ids:
    print("# Time window {}".format(tw_id))
    # pvalue guides
    plt.axhline(0.5, 0, 1, ls=":", color="0.6")
    plt.axhline(1, 0, 1, ls=":", color="0.6")

    best_thresh, best_idx, pvals, scales = res[tw_id]
    plt.axvline(best_thresh, 0, 1, c="#353132", ls="--",
                label=r"$t_\mathrm{{opt}} = {:.2f}$".format(best_thresh))

    plt.plot(threshs[tw_id], pvals, c="0.3", label=r"KS-test $p$", lw=2.5)
    plt.plot(threshs[tw_id], scales, c="0.6", label="Scales", lw=2.5)

    plt.xlim(threshs[tw_id][0], threshs[tw_id][-1])
    plt.xlabel(r"Threshold $t$")
    plt.ylabel("Parameter values")
    plt.title("Time window {}".format(tw_id + 1))
    plt.legend()

    save_fig(plt.gcf(), "plots/bg_ts/bg_ts_thresh_scan_tw_{:2d}.pdf".format(tw_id),
             bbox_inches="tight")
    save_fig(plt.gcf(), "plots/bg_ts/bg_ts_thresh_scan_tw_{:2d}.pgf".format(tw_id),
             bbox_inches="tight")
    plt.show()

Show bg only ts with exp tail

In [ ]:
x = np.linspace(0, max([bg_pdf_i.data.max() for
                        bg_pdf_i in bg_pdfs.values()]), 500)
c_hist, c_emp = "0.65", "#353132"

for tw_id in tw_ids:
    y = bg_pdfs[tw_id].pdf(x, dx=0.25)

    h, bins, err, norm = bg_pdfs[tw_id].data_hist(density=True, dx=0.25,
                                                  which="all")
    m = 0.5 * (bins[:-1] + bins[1:])
    
    # Clip manually for PGF backend
    ymin = np.amin(h[h>0] / 2.)
    h[h < ymin / 10.] = ymin / 10.
    y[y < ymin / 10.] = ymin / 10.

    plt.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color=c_hist)
    plt.errorbar(m, h, yerr=err, fmt=",", color=c_hist, zorder=-10)

    best_thresh = res[tw_id][0]
    plt.axvline(best_thresh, 0, 1, c=c_emp, ls="--")
    plt.plot(x, y, color=c_emp,
             label="Emp. PDF, $t_\mathrm{{opt}}={:.2f}$".format(best_thresh))

    plt.xlabel(r"Test statistic $-2\ln\Lambda$")
    plt.ylabel("PDF")
    plt.xlim(x[0], x[-1])
    plt.ylim(ymin, None)
    plt.yscale("log")
    plt.legend(loc="upper right")
    plt.title("Time window {}".format(tw_id + 1))

    save_fig(plt.gcf(), path="plots/bg_ts/bg_ts_tw_{}.pdf".format(tw_id),
             bbox_inches="tight")
    save_fig(plt.gcf(), path="plots/bg_ts/bg_ts_tw_{:2d}.pgf".format(tw_id),
             bbox_inches="tight")
    plt.show()

Show exp. result with bg only ts - pre trial

In [ ]:
with open(os.path.join(PATHS.local, "unblinding", "result.json")) as fp:
    result = json.load(fp)

In [ ]:
tw_id = 20
x = np.linspace(0, max([bg_pdfs[tw_id].data.max() for
                        bg_pdfs[tw_id] in bg_pdfs.values()]), 500)
c_hist, c_emp = "0.65", "#353132"

y = bg_pdfs[tw_id].pdf(x, dx=0.25)

h, bins, err, norm = bg_pdfs[tw_id].data_hist(density=True, dx=0.25,
                                              which="all")
m = 0.5 * (bins[:-1] + bins[1:])

# Clip manually for PGF backend
ymin = np.amin(h[h>0] / 2.)
h[h < ymin / 10.] = ymin / 10.
y[y < ymin / 10.] = ymin / 10.

plt.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color=c_hist)
plt.errorbar(m, h, yerr=err, fmt=",", color=c_hist, zorder=-10)
plt.plot(x, y, color=c_emp,
         label="Emp. PDF, $t_\mathrm{{opt}}={:.2f}$".format(best_thresh))

plt.axvline(result["ts"][tw_id], 0, 1, c=c_emp, ls="--",
            label=r"Exp. result, " +
            r"$-2\ln\Lambda={:.2f}$".format(result["ts"][tw_id]))


plt.xlabel(r"Test statistic $-2\ln\Lambda$")
plt.ylabel("PDF")
plt.xlim(x[0], x[-1])
plt.ylim(ymin, None)
plt.yscale("log", nonposy="clip")
plt.legend(loc="upper right")
plt.title("Time window {}".format(tw_id + 1))

save_fig(plt.gcf(), path="plots/bg_ts_tw_{}_exp_res.pdf".format(tw_id),
         bbox_inches="tight")  # pdf has problems, because a low negative coord is saved in the pgf, needs to be commented out manually...
save_fig(plt.gcf(), path="plots/bg_ts_tw_{:2d}_exp_res.pgf".format(tw_id),
         bbox_inches="tight")
plt.show()